## Mono-objective versus Multi-objective Algorithms

Visually compares the results generated by the Pareto fronts of the NSGA-II with objective-divided scores of the two EGA and PSO.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from tqdm import tqdm
import datetime
import contextlib
import io
import import_ipynb

import enhanced_genetic_algorithm as ega
import non_dominated_sorting_genetic_algorithm_2 as nsga
import particle_swarm_optimization as pso
import utils.fitness_functions as ff

RUNS = 30

DATASETS = {
    "Small": "data/dataset_small.csv",
    "Medium": "data/dataset_medium.csv",
    "Large": "data/dataset_large.csv"
}

for dataset_label, dataset_file in DATASETS.items():

    dataset_df = pd.read_csv(dataset_file)
    ega.DATASET = dataset_df
    nsga.DATASET = dataset_df
    pso.DATASET = dataset_df

    ga_solutions = []
    pso_solutions = []
    nsga_generations = {}

    with contextlib.redirect_stdout(io.StringIO()):

        for _ in tqdm(range(RUNS), desc=dataset_label):

            timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")

            ega.RUN_TIME = timestamp
            ega.BASE_DIR = f"output/mono_multi_comparison/ega/{dataset_label.lower().replace(' ', '_')}"
            df_ga, best_ga_arrangement = ega.execute()
            ga_div, ga_sat = ff.evaluate_objectives_separately(best_ga_arrangement)
            ga_solutions.append((ga_div, ga_sat))

            pso.RUN_TIME = timestamp
            pso.BASE_DIR = f"output/mono_multi_comparison/pso/{dataset_label.lower().replace(' ', '_')}"
            df_pso, best_pso_arrangement = pso.execute()
            pso_div, pso_sat = ff.evaluate_objectives_separately(best_pso_arrangement)
            pso_solutions.append((pso_div, pso_sat))

            nsga.RUN_TIME = timestamp
            nsga.BASE_DIR = f"output/mono_multi_comparison/nsga_ii/{dataset_label.lower().replace(' ', '_')}"
            pareto_log, _ = nsga.execute_nsga_ii()
            for gen, front in pareto_log.items():
                nsga_generations.setdefault(gen, []).extend(front)

    mean_ga_div = np.mean([div for div, sat in ga_solutions])
    mean_ga_sat = np.mean([sat for div, sat in ga_solutions])

    mean_pso_div = np.mean([div for div, sat in pso_solutions])
    mean_pso_sat = np.mean([sat for div, sat in pso_solutions])

    plt.figure(figsize=(10, 6))

    gens = sorted(nsga_generations.keys())
    cmap = cm.get_cmap("viridis")
    norm = mcolors.Normalize(vmin=min(gens), vmax=max(gens))

    for gen in gens:
        front = nsga_generations[gen]
        divs = [div for div, sat in front]
        sats = [sat for div, sat in front]
        color = cmap(norm(gen))
        label = f"Gen {gen}" if gen % 10 == 0 else None  # only label some gens
        plt.scatter(sats, divs, s=30, alpha=0.6, color=color, label=label)

    # Highlight GA and PSO means
    plt.scatter([mean_ga_sat], [mean_ga_div], color="red", marker="*", s=150, label="Mean Best GA")
    plt.scatter([mean_pso_sat], [mean_pso_div], color="black", marker="X", s=150, label="Mean Best PSO")

    plt.title(f"NSGA-II Pareto Fronts vs. PSO and GA on {dataset_label} Dataset (Mean of {RUNS} Runs)")
    plt.xlabel("Satisfaction Score")
    plt.ylabel("Diversity Score")
    plt.legend(fontsize="small", loc="lower right", ncol=2)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"figures/fig_nsga_vs_ga_pso_{dataset_label.lower()}.png")
    plt.show()